In [3]:
pip install faiss-cpu sentence-transformers numpy


   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
    --------------------------------------- 0.3/18.7 MB ? eta -:--:--
   -- ------------------------------------- 1.0/18.7 MB 2.9 MB/s eta 0:00:07
   --- ------------------------------------ 1.6/18.7 MB 3.1 MB/s eta 0:00:06
   ----- ---------------------------------- 2.4/18.7 MB 3.3 MB/s eta 0:00:05
   ------- -------------------------------- 3.4/18.7 MB 3.6 MB/s eta 0:00:05
   --------- ------------------------------ 4.5/18.7 MB 3.9 MB/s eta 0:00:04
   ------------ --------------------------- 5.8/18.7 MB 4.1 MB/s eta 0:00:04
   -------------- ------------------------- 6.8/18.7 MB 4.3 MB/s eta 0:00:03
   ----------------- ---------------------- 8.1/18.7 MB 4.6 MB/s eta 0:00:03
   -------------------- ------------------- 9.7/18.7 MB 4.9 MB/s eta 0:00:02
   ----------------------- ---------------- 11.0/18.7 MB 5.0 MB/s eta 0:00:02
   -------------------------- ------------- 12.6/18.7 MB 5.3 MB/s eta 0:00:02
   --------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


--> Above cell is for numpy math calculations, vector DB as faiss-cpu, sentence-transformers → to create embeddings (local, free)

In [2]:
# sample_rag.py

documents = [
    """
    Our company offers 20 days of paid annual leave to full-time employees.
    Unused leave can be carried over to the next calendar year up to 10 days.
    """,
    """
    The data platform runs nightly ETL jobs at 1 AM using Apache Airflow.
    These jobs load sales data into the warehouse and refresh dashboards by 7 AM.
    """,
    """
    Employees can work from home up to 3 days per week with manager approval.
    All remote work must comply with security and VPN policies.
    """,
    """
    Code changes must go through pull requests and be approved by at least one reviewer.
    Continuous integration runs unit tests and static code analysis on every commit.
    """
]


--> Above cell is for knowledge base

In [3]:
chunks = []
for i, doc in enumerate(documents):
    chunks.append({
        "id": i,
        "text": doc.strip(),
        "metadata": {
            "source": f"doc_{i}"
        }
    })


--> Above cell is for chunking

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert chunk texts to embeddings
texts = [c["text"] for c in chunks]
embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)

print("Embeddings shape:", embeddings.shape)  # (num_chunks, embedding_dim)


Embeddings shape: (4, 384)


--> Above cell is for embedding

In [5]:
import faiss

# Dimensions = size of embedding vector
dim = embeddings.shape[1]

# Create FAISS index (L2 index; we normalized embeddings so it behaves like cosine)
index = faiss.IndexFlatIP(dim)  # IP = inner product (works well with normalized vectors)

# Add vectors to index
index.add(embeddings)

print("Number of vectors in index:", index.ntotal)


Number of vectors in index: 4


--> Above cell is for vector indexing

In [6]:
def search(query, top_k=2):
    # 1. Embed the query
    q_emb = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)

    # 2. Search in FAISS
    scores, indices = index.search(q_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        results.append({
            "score": float(score),
            "chunk": chunks[int(idx)]
        })
    return results


 --> Above cell Embeds the query, Searches FAISS, Returns top-k chunks

In [10]:
def ask(question, top_k=2):
    print(f"\nQUESTION: {question}")
    results = search(question, top_k=top_k)

    print("\nTop retrieved chunks:\n")
    for r in results:
        print("Score:", round(r["score"], 3))
        print("Source:", r["chunk"]["metadata"]["source"])
        print("Text:\n", r["chunk"]["text"])
        print("-" * 60)


if __name__ == "__main__":
    ask("How many days of annual leave do employees get?")
    #ask("When do the ETL jobs run and which tool is used?")
    #ask("How does code review work in this company?")
    ask("How does employees work in this company?")


QUESTION: How many days of annual leave do employees get?

Top retrieved chunks:

Score: 0.798
Source: doc_0
Text:
 Our company offers 20 days of paid annual leave to full-time employees.
    Unused leave can be carried over to the next calendar year up to 10 days.
------------------------------------------------------------
Score: 0.442
Source: doc_2
Text:
 Employees can work from home up to 3 days per week with manager approval.
    All remote work must comply with security and VPN policies.
------------------------------------------------------------

QUESTION: How does employees work in this company?

Top retrieved chunks:

Score: 0.355
Source: doc_2
Text:
 Employees can work from home up to 3 days per week with manager approval.
    All remote work must comply with security and VPN policies.
------------------------------------------------------------
Score: 0.321
Source: doc_1
Text:
 The data platform runs nightly ETL jobs at 1 AM using Apache Airflow.
    These jobs load sales 

 --> Above cell Rag retrival working or not?